# What is SIR Model

In this notebook I try to explain the SIR framework which is commonly used to model epidemics.

The framework divides the population into categories. 
* S: <b>Susceptible</b> is a group of people who are vulnerable to the infection.
* I: <b>Infectious</b> population is the group of people who can transmit the infection.
* R: <b>Recovered(also called as removed)</b> population is the group of people who are no longer infectious, this includes both the recovered population and people succumbed to the epidemic. 

A key assumption of the model is a recovered person generates immunity towards the epidemic.

The model also assumes that the population remains constant, i.e, there are no new births and no deaths on account of reasons other than the epidemic.

### Initial Conditions

Total population(N) is assumed to be 1. 

Initial state for infectious category(I0) is the fraction of total population which is infectious at time T0.

Initial state for susceptible population(S0) is the remaining population(N-I0) assuming that no person has been vaccinated. 

It is also assumed that there is no recovered person in the beginning. 

### Epidemic starts to spread

<b> New Cases Emerge </b><br>
The epidemic spreads when susceptible person comes into contact with infectious person.<br>
   Number of new infections is proportional to the infection population and susceptible population<br>
   <b>New infection = beta * S * I </b>------(1)
   <br>where beta is the transmission rate<br><br>
   <b>New Recoveries Happen</b><br>
   A fraction of people from the infectious group cease to be infectios as the either die or recover(we will call them recoverd population).<br>Number of recoveries depend on the total number of Infectious people.<br>
   <b>New recovery = gamma * I </b>------(2)<br>where gamma is the recovery rate

### How New Infections and New Recoveries affect S, I and R groups

Susceptible population decreases with new infections.
<br><b>         S[T+1] = S[T] - New Infections[T]</b><br><br>
Infectious population increases with new infections and decreases with new recoveries.
<br><b>         I[T+1] = I[T] + New Infections[T] - New Recovery</b><br><br>
Recovered population increases with new recoveries.
<br><b>         R[T+1] = R[T] +  New Recovery</b>

### Epidemic or not?

An infection turns into an epidemic if it spreads over time i.e, if the new infections is more than the new recoveries.<br><br>
New Infections > New Recoveries<br>
beta \* S \* I > gamma \* I --------- from (1) and (2)<br>
i.e, <b>S * beta / gamma > 1 </b><br>
S \* beta / gamma is called effective reproductive number.<br><br>
At be beginning of epidemic, approximately entire population is susceptible, <br>hence S=S0=1.<br>
Reproduction number = beta / gamma, this is called <b>basic reproduction number, R0</b>.

<br><b>An infection turns into an epidemic if R0 > 1, it dies out if R0 < 1.</b>

# Modelling SIR Framework using Python

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Define Functions

### 1. Function to Model SIR Framework

In [ ]:
def sir_model(I0=0.01, beta=0.6, gamma=0.1):
    """
    Function will take in initial state for infected population,
    Transmission rate (beta) and recovery rate(gamma) as input.
    
    """
    N=1          #Total population
    I=I0         #Initial state of I default value 1% of population
    S=N-I        #Initial state of S
    R=0          #Initial State of R
    C=I          #Initial State of Total Cases
    beta=beta    #Transmission Rate
    gamma=gamma  #Recovery Rate

    inf=[]       # List of Infectious population for each day
    day=[]       # Time period in day
    suc=[]       # List of Susceptible population for each day
    rec=[]       # List of Recovered population for each day
    conf=[]      # List of Total Cases population for each day
    
    for i in range(60):
        day.append(i)
        inf.append(I)
        suc.append(S)
        rec.append(R)
        conf.append(C)

        new_inf= I*S*beta       #New infections equation (1)   
        new_rec= I*gamma        #New Recoveries equation (2)
        
        I=I+new_inf-new_rec     #Total infectious population for next day
        S=S-new_inf             #Total infectious population for next day
        R=R+new_rec             #Total recovered population for next day
        C=C+new_inf             #Total confirmed cases for next day
    
    max_inf=round(np.array(inf).max()*100,2)     #Peak infectious population in percentage
    max_conf=round(np.array(conf).max()*100,2)   #Overal infected population in percentage
    
    print(f"Maximum Infectious population at a time :{max_inf}%")
    print(f"Total Infected population :{max_conf}%")
    
    #Visualizing the model
    sns.set(style="darkgrid")
    plt.figure(figsize=(10,6))
    plt.title(f"SIR Model: R0 = {round(beta/gamma,2)}", fontsize=18)
    sns.lineplot(day,inf, label="Infected")
    sns.lineplot(day,suc,label="Succeptible")
    sns.lineplot(day,rec, label="Recovered")
    #sns.lineplot(day,conf, label="Confirmed") #Generally total infected population is not plotted 
    plt.legend()
    plt.xlabel("Time (in days)")
    plt.ylabel("Fraction of Population")
    plt.show()

### 2. Plot Infectious Population and Total Infected Population for Multiple R0

In [ ]:
def sir_model_betalist(I0=0.01, betalist=[0.5,0.8], gammalist=[0.15,0.25,0.5]):
    """
    Function takes Initial Infected Population(I0), list of transmission rates (betalist)
    and list of recovery rates(gammalist) as arguments.
    Plots Infectious population and Infected Population vs time for input parameters
    """
    
    for gamma in gammalist:
        # Plotting Infectious Population
        plt.figure(figsize=(10,6))
        sns.set(style="darkgrid")
        plt.title("SIR Model: Infectious Population", fontsize=18)
        
        for beta in betalist:
            N=1
            I=I0
            S=N-I
            gamma=gamma
            R0=beta/gamma
            
            inf=[]
            day=[]
            for i in range(50):
                day.append(i)
                inf.append(I)
                new_inf= I*S*beta
                new_rec= I*gamma
                I=I+new_inf-new_rec
                S=S-new_inf
            
            inf_max=round(np.array(inf).max()*100,1)
            sns.lineplot(day,inf, label=f"R0: {round(R0,2)} Peak: {inf_max}%")
            plt.legend()
        plt.show()
        
        # Plotting Total Infected Population
        plt.figure(figsize=(10,6))
        plt.title("SIR Model: Total Confirmed Cases", fontsize=18)       
        for beta in betalist:
            N=1
            I=I0
            S=N-I
            C=I
            gamma=gamma
            R0=beta/gamma
            day=[]
            conf=[]
            for i in range(50):
                day.append(i)
                conf.append(C)

                new_inf= I*S*beta
                new_rec= I*gamma
                I=I+new_inf-new_rec
                S=S-new_inf
                C=C+new_inf
            conf_max=round(np.array(conf).max()*100,1)
            sns.lineplot(day,conf, label=f"R0: {round(R0,2)} Total :{conf_max}%")
            plt.legend()
        plt.show()

## Function Call

### SIR Model for R0=5

R0 for Covid-19 is estimated to be between  2.2 to 5.7 (Source [Forbes Report](https://www.forbes.com/sites/tarahaelle/2020/04/07/the-covid19-coronavirus-disease-may-be-twice-as-contagious-as-we-thought/#7c8b2c1529a6))<br>
We will model a scenario where R0 is 5 and Initial state of infectious population is 1%.<br>
We use beta=0.75 and gamma=0.15 to obtain the desired R0 of 5.0 

In [ ]:
sir_model(beta=0.75,gamma=0.15)

<b>Observation:</b>
* Peak infectious population is 52.43% around 12th day
* Epidemic gets over with entire population getting infected

### SIR Model for R0=3

Now let's examine a case where R0 is reduced to 3.<br> 
Since R0=beta/gamma, R0 reduction is achieved by:
* a. Decreasing beta
* b. Increasing gamma
* c. Both a and b simultaneously

<br>Here we will just reduce beta to 0.45 keeping gamma at previous rate of 0.15 to reduce R0 to 3.

In [ ]:
sir_model(beta=0.45, gamma=0.15)

<b>Observation:</b><br>
* Peak infection population gets reduced from around 50% to around 30%
* Delay in peak of infection, infection peaked at around 12th day when R0=5, whereas it took around 20 days to reach peak of infection when R0 is reduced to 3.
* Around 5% of the total population remains uninfected when the infection ends.

### SIR Model for R0=1.5

Let's explore the nature of epidemic when R0 is furter reduced to 1.5. <br>
We will achieve this by setting gamma=0.3 

In [ ]:
sir_model(beta=0.45,gamma=0.3)

<b>Observation:</b><br>
* Peak level of infectious population is only 7.28% which is much lower than the previous case.
* Infection peaks around 25th day.
* Only around 60% of the total population is infected.

# Flatten the Curve

Flatten the Curve is a phrase which is very familier in these day, this is what we have done above.
It can be observed that, by decreasing R0, we reduce the spread of the infection and the peak level of infected population. Decreasing R0 will also delay the time for peak level of infection, buying us valuable time to prepare tackle the infection effectively.

Let's now check how the peak level of infection and the extent of infection varies with R0.

In [ ]:
sir_model_betalist(I0=0.01,betalist=[0.15,0.2, 0.25,0.30,0.4,0.6,0.8,1,1.2], gammalist=[0.20])

As R0 decreases, the curve for infectious population flattens out.
When R0 is 6, more than 60% of the population is infections by around 8th day, whereas when R0 is 2, maximum infectious population is 16.7% by around 24th day. That mean reduction in R0 from 6 to 2 gives us additional two weeks to prepare to tackle the peak infection. It can also be observed that the extent of infection also decreases with reduction in R0. When R0=6, total population gets infected whereas when R0=2, 20% of the population remains uninfected when the epidemic is over. 

Despite COVID-19 being a relatively less fatal infection, it wreaked havoc all over the world as it quickly spread and caught many countries unprepared, overwhelming their health infrastructure. Health infrastructure is said to be overwhelmed when the number of infectious population is above the total beds available at the hospitals. Yes, the health infrastructure could be ramped up rapidly by converting other infrastructure to makeshift hospitals and calling retired health professionals back into duty etc. as followed by many countries but it won’t be able to keep up with if the cases increase exponentially. The faster the infection curve rises, the quicker the local health care system gets overloaded beyond its capacity to treat people. As we're seeing in Italy, Spain, United States etc. more and more new patients may be forced to go without ICU beds, and more and more hospitals may run out of the basic supplies they need to respond to the outbreak. 

It is in this context "Flattening the Curve" becomes very important.

# How to Flatten the Curve

The curve can be flattened out by reducing R0 which can be achieved by:
*	Reducing susceptible population (S0), 
*	Reducing transmission rate (beta) 
*	Increasing recovery rate (gamma)

<br><b>Reducing Susceptible population</b><br>
The best method for decreasing susceptible population is vaccination, however no vaccination is available for COVID-19. Overall susceptible population can be contained in a region level by implementing travel restrictions, lock-downs etc.
<br><b>Increasing recovery rate (gamma)</b><br>

Increasing recovery rate depends on health infrastructure and treatment methods. Significant improvement in this area is both time consuming and capital intensive.
<br><b>Reduction in transmission rate</b><br>

Transmission rate depends on the number of persons from susceptible population an infected individual comes into contact (<i>contact rate, k</i>) and probability that a contact becomes infected (<i>transmissibility, t</i>).<br> 
Reduce the contact rate by social distancing measures like closing of schools, offices etc., suspension of public transport, self-isolation of susceptible individuals, quarantine etc.<br>
Reduce the transmissibility by improving personal and environmental hygiene by frequent hand washing, covering nose and mouth while sneezing, disinfecting environment, use of face mask etc.
